## Expression Detection

In [4]:
#pip install opencv-python keras

In [5]:
#pip install numpy tensorflow


### Required Libraries

In [6]:
import cv2
import numpy as np
from keras.models import load_model
from keras.optimizers import Adam
import json
import pymongo
from datetime import datetime


### MongoDB connection setup

In [7]:

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["emotion_detection_db"]
collection = db["emotion_results"]


### Loading the model 

In [8]:
def custom_load_model(model_path):
    model = load_model(model_path, compile=False)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model_path = 'fer2013_mini_XCEPTION.102-0.66.hdf5'
model = custom_load_model(model_path)


### Function to detect emotion and return results

In [9]:

# Emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']


def detect_emotion(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)
    
    detected_emotions = []

    for (x, y, w, h) in faces:
        roi_gray = gray_frame[y:y + h, x:x + w]
        roi_gray = cv2.resize(roi_gray, (64, 64))  
        roi_gray = roi_gray.astype('float32') / 255
        roi_gray = np.expand_dims(roi_gray, axis=-1)  
        roi_gray = np.expand_dims(roi_gray, axis=0)  
        preds = model.predict(roi_gray)[0]
        emotion = emotion_labels[np.argmax(preds)]
        detected_emotions.append({
            "emotion": emotion,
            "location": {
                "x": int(x),
                "y": int(y),
                "width": int(w),
                "height": int(h)
            }
        })

    return detected_emotions



### Load and process the image

In [10]:

def process_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Image not found.")
        return
    emotions = detect_emotion(image)
    
    # Create a JSON structure
    result_json = {
        "image_path": image_path,
        "timestamp": datetime.now().isoformat(),
        "detected_emotions": emotions
    }

    # Store in MongoDB
    collection.insert_one(result_json)
    print(f"Results stored in MongoDB for image: {image_path}")


### Usage

In [11]:

image_path = 'smile.jpg' 
process_image(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
Results stored in MongoDB for image: smile.jpg
